<a href="https://colab.research.google.com/github/ykato27/RoBERTa/blob/main/RoBERTa_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# RoBERTa を 動かす
https://huggingface.co/rinna/japanese-roberta-base

- ライブラリ読み込み

In [1]:
# Huggingface Transformersのインストール
!pip install transformers

# Sentencepieceのインストール
!pip install sentencepiece

     |████████████████████████████████| 2.6 MB 8.4 MB/s 
     |████████████████████████████████| 3.3 MB 39.8 MB/s 
     |████████████████████████████████| 636 kB 54.8 MB/s 
     |████████████████████████████████| 895 kB 41.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 9.2 MB/s 


- モデル読み込み

In [2]:
from transformers import T5Tokenizer, RobertaForMaskedLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

model = RobertaForMaskedLM.from_pretrained("rinna/japanese-roberta-base")

Downloading:   0%|          | 0.00/806k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/153 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/259 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/663 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

- 元のテキスト入れる

In [3]:
# original text
text_orig = "ヤマザキパンの祭りは[MASK]に開かれる。"
# text_orig = "くお～！！ぶつかる～！！ここでアクセル全開、[MASK]を右に！"
# text = "4年に1度オリンピックは開かれる。"

# prepend [CLS]
text = "[CLS]" + text_orig

# tokenize
tokens = tokenizer.tokenize(text)
print(tokens)

print('mask index :' , tokens.index('[MASK]'))
# tokens.index('オリンピック')

['[CLS]', '▁', 'ヤマ', 'ザ', 'キ', 'パン', 'の', '祭り', 'は', '[MASK]', '▁', 'に', '開かれる', '。']
mask index : 9


- mask index入れる

In [4]:
# mask a token
masked_idx = 9 # ここにMASKのindexを入れる

- 推論する

In [5]:
tokens[masked_idx] = tokenizer.mask_token
# print(tokens)  # output: ['[CLS]', '▁4', '年に', '1', '度', '[MASK]', 'は', '開かれる', '。']

# convert to ids
token_ids = tokenizer.convert_tokens_to_ids(tokens)
# print(token_ids)  # output: [4, 1602, 44, 24, 368, 6, 11, 21583, 8]

# convert to tensor
import torch
token_tensor = torch.tensor([token_ids])

# get the top 10 predictions of the masked token
model = model.eval()
with torch.no_grad():
    outputs = model(token_tensor)
    predictions = outputs[0][0, masked_idx].topk(10)

print(text_orig)

for i, index_t in enumerate(predictions.indices):
    index = index_t.item()
    token = tokenizer.convert_ids_to_tokens([index])[0]
    print(i, token)

ヤマザキパンの祭りは[MASK]に開かれる。
0 毎週金曜日
1 日曜日
2 土曜日
3 年に一度
4 毎週日曜
5 木曜日
6 金曜日
7 毎週土曜
8 水曜日
9 ▁この時期
